In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pandas as pd
from pyspark.sql import SparkSession

In [4]:
# inicialize session and get context spark
spark = SparkSession.builder.appName("teste").getOrCreate()
sc = spark.sparkContext

Criar um data frame com transações por municípios e data de atualização.

In [5]:
columns = ["transacao","municipio","estado", "data_atualizacao"]
data = [(22000, 'uberlandia', 'MG', '25/02/2021'),
        (25000,'rio de janeiro', 'RJ', '22/02/2021'),
        (27000,'sao paulo', 'SP', '25/02/2021'),
        (35000,'uberlandia', 'MG', '23/02/2021'),
        (1000,'uberlandia', 'MG', '21/02/2021'),
        (122000,'rio de janeiro', 'RJ', '20/02/2021'),
        (200,'belo horizonte', 'MG', '25/02/2021'),
        (800,'sao paulo', 'SP', '24/02/2021'),
        (222,'belo horizonte', 'MG', '22/02/2021'),
        (13000,'rio de janeiro', 'RJ', '23/02/2021'),
        (1000,'sao paulo', 'SP', '25/02/2021'),
        (30000,'uberlandia', 'MG', '21/02/2021')]


In [6]:
schema = StructType([ \
    StructField("transacao",IntegerType(),True), \
    StructField("municipio",StringType(),True), \
    StructField("estado",StringType(),True), \
    StructField("data_atualizacao", StringType(), True),
  ])

In [7]:
df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- transacao: integer (nullable = true)
 |-- municipio: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_atualizacao: string (nullable = true)

+---------+--------------+------+----------------+
|transacao|municipio     |estado|data_atualizacao|
+---------+--------------+------+----------------+
|22000    |uberlandia    |MG    |25/02/2021      |
|25000    |rio de janeiro|RJ    |22/02/2021      |
|27000    |sao paulo     |SP    |25/02/2021      |
|35000    |uberlandia    |MG    |23/02/2021      |
|1000     |uberlandia    |MG    |21/02/2021      |
|122000   |rio de janeiro|RJ    |20/02/2021      |
|200      |belo horizonte|MG    |25/02/2021      |
|800      |sao paulo     |SP    |24/02/2021      |
|222      |belo horizonte|MG    |22/02/2021      |
|13000    |rio de janeiro|RJ    |23/02/2021      |
|1000     |sao paulo     |SP    |25/02/2021      |
|30000    |uberlandia    |MG    |21/02/2021      |
+---------+--------------+------+----------------+



Considerando o data frame agrupar por município e considerando a data de atualização definir a ordem da transação.

In [9]:
df = df.groupby(col('municipio'),col('estado'), col('data_atualizacao')) \
        .sum() \
        .orderBy(col('data_atualizacao')) \
        .withColumnRenamed("sum(transacao)", 'transacao')

In [8]:
df = df.withColumn("data_atualizacao", unix_timestamp(col("DATA_ATUALIZACAO"), 'dd/mm/yyyy').cast('timestamp'))

In [11]:
df.printSchema()
df.show(truncate=False)

root
 |-- municipio: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_atualizacao: timestamp (nullable = true)
 |-- transacao: long (nullable = true)

+--------------+------+-------------------+---------+
|municipio     |estado|data_atualizacao   |transacao|
+--------------+------+-------------------+---------+
|rio de janeiro|RJ    |2021-01-20 00:02:00|122000   |
|uberlandia    |MG    |2021-01-21 00:02:00|31000    |
|belo horizonte|MG    |2021-01-22 00:02:00|222      |
|rio de janeiro|RJ    |2021-01-22 00:02:00|25000    |
|uberlandia    |MG    |2021-01-23 00:02:00|35000    |
|rio de janeiro|RJ    |2021-01-23 00:02:00|13000    |
|sao paulo     |SP    |2021-01-24 00:02:00|800      |
|belo horizonte|MG    |2021-01-25 00:02:00|200      |
|sao paulo     |SP    |2021-01-25 00:02:00|28000    |
|uberlandia    |MG    |2021-01-25 00:02:00|22000    |
+--------------+------+-------------------+---------+



In [12]:
#Rodando todas operações em um comando.

In [10]:
df = df.withColumn("data_atualizacao", unix_timestamp(col("DATA_ATUALIZACAO"), 'dd/mm/yyyy').cast('timestamp')).groupby(col('municipio'),col('estado'), col('data_atualizacao')) \
        .sum() \
        .orderBy(col('data_atualizacao')) \
        .withColumnRenamed("sum(transacao)", 'transacao')
df.printSchema()
df.show(truncate=False)

root
 |-- municipio: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_atualizacao: timestamp (nullable = true)
 |-- transacao: long (nullable = true)

+--------------+------+-------------------+---------+
|municipio     |estado|data_atualizacao   |transacao|
+--------------+------+-------------------+---------+
|rio de janeiro|RJ    |2021-01-20 00:02:00|122000   |
|uberlandia    |MG    |2021-01-21 00:02:00|31000    |
|belo horizonte|MG    |2021-01-22 00:02:00|222      |
|rio de janeiro|RJ    |2021-01-22 00:02:00|25000    |
|rio de janeiro|RJ    |2021-01-23 00:02:00|13000    |
|uberlandia    |MG    |2021-01-23 00:02:00|35000    |
|sao paulo     |SP    |2021-01-24 00:02:00|800      |
|belo horizonte|MG    |2021-01-25 00:02:00|200      |
|uberlandia    |MG    |2021-01-25 00:02:00|22000    |
|sao paulo     |SP    |2021-01-25 00:02:00|28000    |
+--------------+------+-------------------+---------+



O data frame final precisa conter um campo ordenando as transações por município
- Transação, Município, Estado, data de Atualização e Ordem da Transação

In [13]:
w = Window().orderBy("MUNICIPIO")
df2 = df.withColumn("ORDEM_TRANSACAO", row_number().over(w))

In [14]:
df2.printSchema()
df2.show(truncate=False)

root
 |-- municipio: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_atualizacao: timestamp (nullable = true)
 |-- transacao: long (nullable = true)
 |-- ORDEM_TRANSACAO: integer (nullable = true)

+--------------+------+-------------------+---------+---------------+
|municipio     |estado|data_atualizacao   |transacao|ORDEM_TRANSACAO|
+--------------+------+-------------------+---------+---------------+
|belo horizonte|MG    |2021-01-22 00:02:00|222      |1              |
|belo horizonte|MG    |2021-01-25 00:02:00|200      |2              |
|rio de janeiro|RJ    |2021-01-20 00:02:00|122000   |3              |
|rio de janeiro|RJ    |2021-01-22 00:02:00|25000    |4              |
|rio de janeiro|RJ    |2021-01-23 00:02:00|13000    |5              |
|sao paulo     |SP    |2021-01-24 00:02:00|800      |6              |
|sao paulo     |SP    |2021-01-25 00:02:00|28000    |7              |
|uberlandia    |MG    |2021-01-21 00:02:00|31000    |8              |
|uberla

In [ ]:
#ordenacao via sql costumo utilizar mais

In [13]:
df.createOrReplaceTempView("transmun")

In [30]:
dfsql = spark.sql("SELECT municipio, estado, data_atualizacao, transacao, ROW_NUMBER() OVER(ORDER BY municipio ASC) as ordem_transacao FROM transmun")
dfsql.show()

+--------------+------+-------------------+---------+---------------+
|     municipio|estado|   data_atualizacao|transacao|ordem_transacao|
+--------------+------+-------------------+---------+---------------+
|belo horizonte|    MG|2021-01-22 00:02:00|      222|              1|
|belo horizonte|    MG|2021-01-25 00:02:00|      200|              2|
|rio de janeiro|    RJ|2021-01-20 00:02:00|   122000|              3|
|rio de janeiro|    RJ|2021-01-22 00:02:00|    25000|              4|
|rio de janeiro|    RJ|2021-01-23 00:02:00|    13000|              5|
|     sao paulo|    SP|2021-01-24 00:02:00|      800|              6|
|     sao paulo|    SP|2021-01-25 00:02:00|    28000|              7|
|    uberlandia|    MG|2021-01-21 00:02:00|    31000|              8|
|    uberlandia|    MG|2021-01-23 00:02:00|    35000|              9|
|    uberlandia|    MG|2021-01-25 00:02:00|    22000|             10|
+--------------+------+-------------------+---------+---------------+

